<a href="https://colab.research.google.com/github/jimin0/2023_JUMP_AI/blob/main/%EC%8B%A0%EC%95%BD%EA%B0%9C%EB%B0%9C_deep_learning_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-z0fxqn8t
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-z0fxqn8t
  Resolved https://github.com/huggingface/transformers to commit c3ecf2d95d6a9f614d968af2f8b4e317f381e5ec
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install dill

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATA_PATH = '/content/drive/MyDrive/2023 신약개발 대회/open (1)/train.csv'
SUBMISSION_PATH = '/content/drive/MyDrive/2023 신약개발 대회/open (1)/test.csv'

class ModelCfg:
  epochs = 10
  seed = 42
  n_fold = 5
  batch_size=24

  MODEL_NAME = 'seyonec/ChemBERTa-zinc-base-v1'
  MAX_SEQ_LENGTH = 0
  TOKENIZER = None
  MODEL_PATH_HLM = '/content/drive/MyDrive/2023 신약개발 대회/신약개발 코드/model_HLM'
  MODEL_PATH_MLM = '/content/drive/MyDrive/2023 신약개발 대회/신약개발 코드/model_MLM'
  SAVE_PATH_HLM = '/content/drive/MyDrive/2023 신약개발 대회/신약개발 코드/HLM_config.pth'
  SAVE_PATH_MLM = '/content/drive/MyDrive/2023 신약개발 대회/신약개발 코드/HLM_config.pth'
  CLEAN_DATA_PATH = '/content/drive/MyDrive/2023 신약개발 대회/신약개발 코드/cleaned'
  CONFIG_PATH_HLM = MODEL_PATH_HLM + 'model_contig.pth'
  CONFIG_PATH_MLM = MODEL_PATH_MLM + 'model_contig.pth'

In [ ]:
import numpy as np
import pandas as pd
import random
import os
import matplotlib.pyplot as plt
import json
from ast import literal_eval
from tqdm.auto import tqdm
tqdm.pandas()
import re
import sys
import copy
import time
import math
import string
import pickle
import joblib
import itertools
import platform
import collections
import scipy as sp
import gc
import warnings; warnings.filterwarnings("ignore")
from scipy.stats import gmean

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

import tokenizers
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModel
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

print('torch version : ' , torch.__version__)
print('tokenizers version : ', tokenizers.__version__)
print('transformers version : ', transformers.__version__)
%env TOKENIZERS_PARALLELISM=true

torch version :  2.0.1+cu118
tokenizers version :  0.14.0
transformers version :  4.34.0.dev0
env: TOKENIZERS_PARALLELISM=true


In [ ]:
if not os.path.exists(ModelCfg.MODEL_PATH_HLM):
  print(f'create directory...[{ModelCfg.MODEL_PATH_HLM}]')
  os.mkdir(ModelCfg.MODEL_PATH_HLM)
if not os.path.exists(ModelCfg.MODEL_PATH_MLM):
  print(f'create directory...[{ModelCfg.MODEL_PATH_MLM}]')
  os.mkdir(ModelCfg.MODEL_PATH_MLM)
if not os.path.exists(ModelCfg.CLEAN_DATA_PATH):
  print(f'create dircetory...[{ModelCfg.CLEAN_DATA_PATH}]')
  os.mkdir(ModelCfg.CLEAN_DATA_PATH)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device: ', device)

device:  cuda


In [ ]:
def RMSELoss(yhat, y): # metric함수
  return torch.sqrt(torch.mean((yhat-y)**2))

In [ ]:
def reset_seeds(seed): # seed 고정을 위한 함수
  random.seed(seed)
  os.environ['PATHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)

reset_seeds(ModelCfg.seed)

In [ ]:
def read_data_list():
    train = pd.read_csv(f'{DATA_PATH}')
    test = pd.read_csv(f'{SUBMISSION_PATH}')

    print('train.shape: ', train.shape)
    print('test.shape: ', test.shape)

    return train, test

train, test = read_data_list()

train.shape:  (3498, 11)
test.shape:  (483, 9)


In [ ]:
def make_tokenizer(): # tokenizer를 반환하는 함수
  tokenizer = AutoTokenizer.from_pretrained(f'{ModelCfg.MODEL_NAME}', normalization=True, caeche_dir='/content/drive/MyDrive/2023 신약개발 대회/신약개발 코드/cache')
  tokenizer.save_pretrained(f'{ModelCfg.MODEL_PATH_HLM}')
  tokenizer.save_pretrained(f'{ModelCfg.MODEL_PATH_MLM}')
  print(f'tokenizer object load & save...[{ModelCfg.MODEL_NAME}]')

  example = "Cn1c(=O)c2c(ncn2C)n(C)c1=O"
  tokens = tokenizer(example)
  tokens_ = tokenizer.tokenize(example)
  print(tokens)
  print(tokens_)

  return tokenizer

tokenizer = make_tokenizer()
ModelCfg.TOKENIZER = tokenizer

tokenizer object load & save...[seyonec/ChemBERTa-zinc-base-v1]
{'input_ids': [0, 308, 21, 71, 263, 51, 13, 71, 22, 71, 12, 371, 22, 39, 13, 82, 12, 39, 13, 71, 21, 33, 51, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['Cn', '1', 'c', '(=', 'O', ')', 'c', '2', 'c', '(', 'ncn', '2', 'C', ')', 'n', '(', 'C', ')', 'c', '1', '=', 'O']


In [ ]:
def cacl_max_length(train_data): # 어짜피 dataset은 둘다 같기 때문에 HLM의 max_len을 구한다.
  print(f'calculate max sequence length...[{ModelCfg.MODEL_NAME}]')
  for text_col in ['SMILES']:
    SMILES_lengths = []
    text_values = tqdm(train[text_col].fillna('').values, total=len(train))
    for text in text_values:
      length = len(ModelCfg.TOKENIZER(text, add_special_tokens=False)['input_ids'])
      SMILES_lengths.append(length)

    max_seq_length = max(SMILES_lengths) + 2
    print('SMILES max length : ', max(SMILES_lengths))
    print('final max data length : ', max_seq_length)

    return max_seq_length

max_seq_length = cacl_max_length(train)
ModelCfg.Max_SEQ_LENGTH = max_seq_length

calculate max sequence length...[seyonec/ChemBERTa-zinc-base-v1]


  0%|          | 0/3498 [00:00<?, ?it/s]

SMILES max length :  123
final max data length :  125


In [ ]:
def prepare_input(tokenizer, max_seq_length, smiles):
  inputs = tokenizer(smiles, add_special_tokens=True, truncation=True, max_length=max_seq_length, padding='max_length', return_offsets_mapping=False)
  for k, v in inputs.items():
    inputs[k] = torch.tensor(v, dtype=torch.long)
  return inputs

def create_label(label):
  return torch.tensor([label], dtype=torch.float)

class myDataset(Dataset): # dataset을 token화하고 train이라면 label값 까지 반환
  def __init__(self, cfg, df, is_train=False, labeling='HLM'):
    self.cfg = cfg
    self.is_train = is_train
    self.labeling = labeling
    self.smiles = df['SMILES'].values
    if not self.is_train:
      if self.labeling == 'HLM':
        self.HLM = df['HLM'].values
      else:
        self.MLM = df['MLM'].values

  def __len__(self):
    return len(self.smiles)

  def __getitem__(self, item):
    inputs = prepare_input(self.cfg.TOKENIZER, self.cfg.MAX_SEQ_LENGTH, self.smiles[item])
    label = np.array([])
    if not self.is_train:
      if self.labeling == 'HLM':
        label = create_label(self.HLM[item])
      else:
        label = create_label(self.MLM[item])
    return inputs, label

In [ ]:
# 작동 확인
temp_train_dataset = myDataset(ModelCfg, train)
temp_train_loader = DataLoader(temp_train_dataset, batch_size=1, shuffle=False, drop_last=True)
inputs, labels = next(iter(temp_train_loader))
print('inputs : ', inputs)
print('labels : ', labels.shape, labels)

inputs :  {'input_ids': tensor([[  0, 319,  21, 264,  12, 300, 263,  51,  13,  71,  22, 261, 283,  71,
          23, 296,  12,  39,  13, 270,  23,  39,  13,  82,  63, 290,  65,  22,
          13, 261,  21, 323,   2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1]])}
labels :  torch.Size([1, 1]) tensor([[50.6800]])


In [ ]:
class myModel(nn.Module): # smiles data로 MLM HLM 각각 한가지씩 추출
  def __init__(self, cfg, config_path=None, pretrained=False):
    super().__init__()
    self.cfg = cfg
    if config_path is None:
      self.config = AutoConfig.from_pretrained(cfg.MODEL_NAME, output_hidden_states = True)
    else:
      self.config = torch.load(config_path)
    if pretrained:
      self.model = AutoModel.from_pretrained(cfg.MODEL_NAME, config=self.config, cache_dir='/content/drive/MyDrive/2023 신약개발 대회/신약개발 코드/cache')
    else:
      self.model = AutoModel.from_config(self.config)

    self.fc_dropout = nn.Dropout(0.2)
    self.fc = nn.Linear(self.config.hidden_size, 1)

  def feature(self, inputs):
      outputs = self.model(**inputs)
      last_hidden_states = outputs[0]
      return last_hidden_states

  def forward(self, inputs):
      feature = self.feature(inputs)
      output = self.fc(self.fc_dropout(feature))
      return output

In [ ]:
def get_optimizer_params(model): ## 학습률 조정
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

    return optimizer_parameters


In [ ]:
def get_scheduler(cfg, optimizer, num_train_steps):
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)
    return scheduler

In [ ]:
class LoseMeter(object):
    def __init__(self):
        self.reset()
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
def train_fn(train_loader, model, criterion, optimizer, epoch, scheduler, device):
  model.train()
  scaler = torch.cuda.amp.GradScaler(enabled=True)
  losses = LoseMeter()
  epoch_display = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
  epoch_display.set_description(f'Epoch[{epoch+1}/{ModelCfg.epochs}]')
  for step, (inputs, labels) in epoch_display:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        y_preds = model(inputs)
        y_preds = y_preds[::, 0]
        loss = criterion(y_preds, labels)
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        scheduler.step()
        epoch_display.set_postfix(train_loss=losses.avg, lr=scheduler.get_lr()[0])

  return losses.avg

def inference_fn(test_loader, model, device):
    preds = []
    model.eval()
    model.to(device)
    input_values = tqdm(test_loader, total=len(test_loader))
    for inputs, _ in input_values:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        y_preds = y_preds[::, 0]
        preds.append(y_preds.to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

In [ ]:
def train_loop(train):
  # data Load
  x_train_ = train.copy()
  train_dataset_HLM = myDataset(ModelCfg, x_train_, labeling='HLM')
  train_dataset_MLM = myDataset(ModelCfg, x_train_, labeling='MLM')
  train_loader_HLM = DataLoader(train_dataset_HLM, batch_size=ModelCfg.batch_size, shuffle=True)
  train_loader_MLM = DataLoader(train_dataset_MLM, batch_size=ModelCfg.batch_size, shuffle=True)

  # model
  model_HLM = myModel(ModelCfg, config_path=None, pretrained=True)
  model_MLM = myModel(ModelCfg, config_path=None, pretrained=True)
  torch.save(model_HLM.config, ModelCfg.SAVE_PATH_HLM)
  torch.save(model_MLM.config, ModelCfg.SAVE_PATH_MLM)
  model_HLM.to(device)
  model_MLM.to(device)

  # optimizer
  optimizer_parameters_HLM = get_optimizer_params(model_HLM)
  optimizer_HLM = AdamW(optimizer_parameters_HLM, lr=2e-5, eps=1e-8, betas=(0.9, 0.999))
  optimizer_parameters_MLM = get_optimizer_params(model_MLM)
  optimizer_MLM = AdamW(optimizer_parameters_MLM, lr=2e-5, eps=1e-8, betas=(0.9, 0.999))

  # scheduler
  num_train_steps1 = int(len(x_train_) / ModelCfg.batch_size * ModelCfg.epochs)
  num_train_steps2 = int(len(x_train_) / ModelCfg.batch_size * ModelCfg.epochs)
  scheduler_HLM = get_scheduler(ModelCfg, optimizer_HLM, num_train_steps1)
  scheduler_MLM = get_scheduler(ModelCfg, optimizer_MLM, num_train_steps2)

  # loop
  criterion = RMSELoss
  best_score_HLM = np.inf
  best_score_MLM = np.inf
  for epoch in range(ModelCfg.epochs):
      start_time = time.time()

      # train
      avg_train_HLM_loss = train_fn(train_loader_HLM, model_HLM, criterion, optimizer_HLM, epoch, scheduler_HLM, device)
      avg_train_MLM_loss = train_fn(train_loader_MLM, model_MLM, criterion, optimizer_MLM, epoch, scheduler_MLM, device)

      # scoring
      epoch_display_HLM = tqdm([0], leave=True)
      epoch_display_MLM = tqdm([0], leave=True)

      for _ in epoch_display_HLM:
          epoch_display_HLM.set_description(f'Epoch [{epoch+1}/{ModelCfg.epochs}]')

          score_HLM = avg_train_HLM_loss

          if best_score_HLM > score_HLM:
              best_score_HLM = score_HLM
              torch.save({'model_HLM': model_HLM.state_dict()}, f"{ModelCfg.MODEL_PATH_HLM}{ModelCfg.MODEL_NAME.replace('/', '-')}_best.pth")


      for _ in epoch_display_MLM:
          epoch_display_MLM.set_description(f'Epoch [{epoch+1}/{ModelCfg.epochs}]')

          score_MLM = avg_train_MLM_loss

          if best_score_MLM > score_MLM:
              best_score_MLM = score_MLM
              torch.save({'model_MLM': model_MLM.state_dict()}, f"{ModelCfg.MODEL_PATH_MLM}{ModelCfg.MODEL_NAME.replace('/', '-')}_best.pth")
              print(1)

          elapsed = time.time() - start_time
          epoch_display_HLM.set_postfix(avg_train_HLM_loss=f'{avg_train_HLM_loss:.4f}', time=f'{elapsed/60:.0f}m', Score=f'{score_HLM:.4f}')
          epoch_display_MLM.set_postfix(avg_train_MLM_loss=f'{avg_train_MLM_loss:.4f}', time=f'{elapsed/60:.0f}m', Score=f'{score_MLM:.4f}')

torch.cuda.empty_cache()
gc.collect()

5321

In [ ]:
# 데이타 읽기
print('read data.............')
train, test = read_data_list()

# tokenizer 생성
print('create tokenizer.............')
tokenizer = make_tokenizer()
ModelCfg.TOKENIZER = tokenizer

# tokenizer에 입력하는 최대 길이 계산
print('calculate max sequence length.............')
max_seq_length = cacl_max_length(train)
ModelCfg.MAX_SEQ_LENGTH = max_seq_length

# 훈련
print(f'TRAINING [{ModelCfg.MODEL_NAME}] MODEL.............')
train_loop(train)

# 예측
print(f'INFERENCE [{ModelCfg.MODEL_NAME}] MODEL.............')
test_dataset_HLM = myDataset(ModelCfg, test, True, 'HLM')
test_dataset_MLM = myDataset(ModelCfg, test, True, 'MLM')
test_loader = DataLoader(test_dataset_HLM, batch_size=ModelCfg.batch_size, shuffle=False, drop_last=False)
test_loader = DataLoader(test_dataset_MLM, batch_size=ModelCfg.batch_size, shuffle=False, drop_last=False)
model_HLM = myModel(ModelCfg, config_path=None, pretrained=True)
model_MLM = myModel(ModelCfg, config_path=None, pretrained=True)
state_HLM = torch.load(f"{ModelCfg.MODEL_PATH_HLM}{ModelCfg.MODEL_NAME.replace('/', '-')}_best.pth", map_location=torch.device('cpu'))
state_MLM = torch.load(f"{ModelCfg.MODEL_PATH_MLM}{ModelCfg.MODEL_NAME.replace('/', '-')}_best.pth", map_location=torch.device('cpu'))
model_HLM.load_state_dict(state_HLM['model_HLM'])
model_MLM.load_state_dict(state_MLM['model_MLM'])
prediction_HLM = inference_fn(test_loader, model_HLM, device)
prediction_MLM = inference_fn(test_loader, model_MLM, device)
del model_HLM, model_MLM, state_HLM, state_MLM
gc.collect()
torch.cuda.empty_cache()

print(f'[{ModelCfg.MODEL_NAME}] predictions shape : ', np.array(prediction_HLM).shape)


read data.............
train.shape:  (3498, 11)
test.shape:  (483, 9)
create tokenizer.............
tokenizer object load & save...[seyonec/ChemBERTa-zinc-base-v1]
{'input_ids': [0, 308, 21, 71, 263, 51, 13, 71, 22, 71, 12, 371, 22, 39, 13, 82, 12, 39, 13, 71, 21, 33, 51, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['Cn', '1', 'c', '(=', 'O', ')', 'c', '2', 'c', '(', 'ncn', '2', 'C', ')', 'n', '(', 'C', ')', 'c', '1', '=', 'O']
calculate max sequence length.............
calculate max sequence length...[seyonec/ChemBERTa-zinc-base-v1]


  0%|          | 0/3498 [00:00<?, ?it/s]

SMILES max length :  123
final max data length :  125
TRAINING [seyonec/ChemBERTa-zinc-base-v1] MODEL.............


  0%|          | 0/146 [00:00<?, ?it/s]

  0%|          | 0/146 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

1
INFERENCE [seyonec/ChemBERTa-zinc-base-v1] MODEL.............


  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/21 [00:00<?, ?it/s]

[seyonec/ChemBERTa-zinc-base-v1] predictions shape :  (483, 1)


In [ ]:
submission_df = pd.read_csv('/content/drive/MyDrive/2023 신약개발 대회/open (1)/sample_submission.csv', encoding='UTF-8')
submission_df['HLM'] = prediction_HLM
submission_df['MLM'] = prediction_MLM
fname = '/content/drive/MyDrive/2023 신약개발 대회/submission/Submission_Smiles_ver2.csv'
submission_df.to_csv(fname, index=False)